In [34]:
import pandas as pd
import numpy as np
import math
import csv
import re
import json
import time
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions 
from pyspark.sql import Row
from collections import OrderedDict


In [35]:
# parsing data from sgm file

from bs4 import BeautifulSoup,SoupStrainer


def readData(fileName):
    
    data = ''
    f = open(fileName, 'rb')
    soup = BeautifulSoup(f, 'html.parser')
    # get body content
    contents = soup.findAll('body')
    
    return contents

# parsing data from sgm file

In [36]:
# Q1

# ==== initial setting ====

sTime = time.time()
file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
bodyLen, shingleLen = 0, 0
# storage for Shingle
twoShingleSet = set()

# ==== initial setting ====

print("Start count shingle")
for i in range(len(file)):
    fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
    fileName = fileName + file[i] + ".sgm"
    # read body content from each file
    data = readData(fileName)
    # retrival all body content
    for content in data:
        if i < 2:
            bodyLen +=1
        # split one body content by \n
        for d in content.get_text().split("\n"):
            # remove header space and footer space
            sentence = d.strip()
            # regular expresisson
            pat = '[a-zA-Z]+'
            reContent = re.findall(pat, sentence)
            # select two shingle on each time
            for ind in range(len(reContent)-1):
                shingle = reContent[ind] + ' ' + reContent[ind+1]
                # add shingle
                twoShingleSet.add(shingle)
    print("file ", i, " done")
    
shingleLen = len(twoShingleSet)
print("Count shingle done\n")


print("Total body number: %d" % bodyLen)
print("Total shingle number: %d" % shingleLen)
print("Count shingle took: %.2fs" % (time.time()-sTime))

# Q1


Start count shingle
file  0  done
file  1  done
file  2  done
file  3  done
file  4  done
file  5  done
file  6  done
file  7  done
file  8  done
file  9  done
file  10  done
file  11  done
file  12  done
file  13  done
file  14  done
file  15  done
file  16  done
file  17  done
file  18  done
file  19  done
file  20  done
file  21  done
Count shingle done

Total body number: 1855
Total shingle number: 520470
Count shingle took: 41.95s


In [ ]:
# # Q1

# ==== initial setting ====

sTime = time.time()
file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
trueNum = 0
pat = '[a-zA-Z]+'
matrix = np.full((shingleLen,bodyLen), False, dtype=bool)

# ==== initial setting ====


print("Start calculate shingle matrix")
for i in range(2):
    fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
    fileName = fileName + file[i] + ".sgm"
    print(fileName +" start!")
    # read body content from each file
    data = readData(fileName)
    # save matrix result for each body
    col = 0
    contInd = 0
    for content in data:
        # save shingle from one body content
        bContShingleSet = set()
        # split one body content by \n
        for d in content.get_text().split("\n"):
            # remove header space and footer space
            sentence = d.strip()
            # regular expresisson
            reContent = re.findall(pat, sentence)
            # select two shingle on each time
            for r in range(len(reContent)-1):
                shingle = reContent[r] + ' ' + reContent[r+1]
                bContShingleSet.add(shingle)
        row = 0
        for t in twoShingleSet:
            if t in bContShingleSet:
                matrix[row][col] = True
                trueNum += 1
            row +=1
        col +=1
        contInd +=1
        if contInd % 100 ==0:
            print(contInd, " done!")
            
print("total trueNum ", trueNum)
print("Calculate shingle matrix done")
print("Calculate shingle matrix took %.2fs" % (time.time()-sTime))

# save Q1 Result      
sTime = time.time()
np.savetxt("Q1_Result.csv", matrix, delimiter=",")
print("Save shingle matrix took %.2f" % (time.time()-sTime))



# Q1

Start calculate shingle matrix
/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-000.sgm start!
100  done!
200  done!
300  done!
400  done!
500  done!
600  done!
700  done!
800  done!
900  done!
/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-001.sgm start!
100  done!
200  done!
300  done!
400  done!
500  done!
600  done!
700  done!
800  done!
900  done!
total trueNum  183231
Calculate shingle matrix done
Calculate shingle matrix took 220.15s
Save shingle matrix took 547.37


In [ ]:

# Q2

# ==== initial setting ====

sTime = time.time()
index = 0
flag = 0
minHash = np.zeros(bodyLen,int)
idx = np.full(shingleLen, False, dtype=bool)
curNum = 0
ind = 0
mod = 10

# ==== initial setting ====

print("Start process minHash!")
while index < shingleLen and flag < bodyLen:
    curNum = ind
    index +=1
    if(curNum + mod)< shingleLen:
        ind = curNum + mod
    else:
        ind = (curNum + mod) % shingleLen
    while idx[ind] == True:
        ind += 1
    idx[ind] = True
    for c in range(bodyLen):
        if matrix[ind][c] == True:
            if minHash[c] == 0:
                minHash[c] = index
                flag +=1
    if index % 2000 ==0:
        print("flag ",flag, " done!")
        print("index ", index, " done!")
        
print("Process minHash done!")
print("MinHash calculate took %.2f" % (time.time()-sTime))

# save Q2 Result
sTime = time.time()
np.savetxt("Q2_Result.csv", minHash, delimiter=",")
print("Save minHash matrix took %.2fs" % (time.time()-sTime))


# Q2


Start process minHash!
flag  375  done!
index  2000  done!
flag  605  done!
index  4000  done!
flag  795  done!
index  6000  done!
flag  851  done!
index  8000  done!
flag  884  done!
index  10000  done!
flag  894  done!
index  12000  done!
flag  903  done!
index  14000  done!
flag  915  done!
index  16000  done!
flag  917  done!
index  18000  done!
flag  918  done!
index  20000  done!
flag  921  done!
index  22000  done!
flag  923  done!
index  24000  done!
flag  923  done!
index  26000  done!
flag  925  done!
index  28000  done!
flag  927  done!
index  30000  done!
flag  927  done!
index  32000  done!
flag  927  done!
index  34000  done!
flag  928  done!
index  36000  done!
flag  929  done!
index  38000  done!
flag  929  done!
index  40000  done!
flag  929  done!
index  42000  done!
flag  929  done!
index  44000  done!
flag  930  done!
index  46000  done!
flag  930  done!
index  48000  done!
flag  930  done!
index  50000  done!
flag  930  done!
index  52000  done!
flag  930  done!
in